In [1]:
import dao.MongoUsersSong as mus
import panda.SongUser as supd 
# 重要信息，此程序需要再user_tag_recom_main.py 和 user_song_recom_main.py后运行
# 通过用户进行歌曲的歌曲相似度推荐 获取的其中的相似歌曲
mongomus = mus.UserSongRecomAfterSongSimilarity(
    user_id="1a78ac10b67911ed95dd00155dadb10b")
# 1a78ac10b67911ed95dd00155dadb10b
# mongomus.makeRecomBySongSimilarityAnswer(limit="ONE")
# 创建程序进程

In [2]:
# 程序从这里开始
# ONE 表示只操作目标用户，ALL 时 
# 表示目标用户的id 不再重要，会记录所有用户的歌曲之间的相似度
# 同时ALL时 是将数据存储到song 中，记录为SimilaritySongs
# ONE，是将数据存储到recom 中，记录为RecomSong
# mongomus.makeRecomBySongSimilarityAnswer(limit="ONE")
# 此处暂时使用ONE 标签的程序过程
# 此处sign 适用于判断是输出哪种格式
# mongomus.makeRecomAnswerSong(sign=1)
# 首先是ONE 和 ALL 两种模式的共同方法

# 获取recom中之前设置的数据
projections={"_id": 0, "id": 1, "name": 1, "recomUsers": 1}
doc=mongomus.rdb.findDocument(
        collection_name = "recom", query = mongomus.query, projection = projections)
# 储存请求用户的名字
mongomus.usr.user_name=doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id=[]
list_users_id.append(mongomus.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
# print(doc)
list_users_id.extend(doc[0]['recomUsers'])
# 此处生成list_users_id
print(len(list_users_id))
for item in list_users_id:
    print(item)
    break


8
1a78ac10b67911ed95dd00155dadb10b


In [3]:
# 构成xy轴的内容源头
docs=mongomus.usr.getSongsWithUserIdFromMongo(array = list_users_id)

In [4]:
# xy稀疏矩阵信息,显示docs 的长度
# print(type(docs))
print(len(docs))


8


In [5]:
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data=mongomus.usr.makeXYMatric(docs = docs)
# data 存储的是每个歌曲信息的坐标位置信息
# for item in data:
#     print(data)
#     break
# 生成pandas
sudf=supd.SUPandas(datas = list(
    map(list, data.toarray())), items = mongomus.usr.y, users = mongomus.usr.x)

In [6]:
# 最后构成的稀疏矩阵
sudf.df

,RuRuUIH,m3NOW7Pp,UnfVc1Tg,HY0HwB,fhHz,9E,zVZzhaw,FnJi
35504129,1,0,0,0,0,0,0,0
253956,0,1,0,0,0,0,0,0
1330364421,0,0,0,1,0,0,0,0
552869896,0,0,0,0,1,0,0,0
29097995,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...
401396,1,0,0,0,0,0,0,0
1294322,0,0,0,1,0,0,0,0
28221434,0,1,0,0,0,0,0,0
27598844,0,0,0,0,0,0,0,1


In [7]:
# 寻找用户的推荐歌曲
projections = {"_id": 0, "songs": 1}
# dict_song = mongomus.makeRecomSongDictionForUser(
#     df_object=sudf, user_songs=mongomus.rdb.findDocument(
#         collection_name="like", query=mongomus.query, projection=projections)[0]['songs'])
# 获得目标用户的songs
user_songs = mongomus.rdb.findDocument(collection_name="like", query=mongomus.query, projection=projections)
# 展示第一条数据
print(user_songs[0]['songs'][0])
user_songs = user_songs[0]['songs']

{'id': 1370027704, 'name': '复甦（中式医乐·黄钟宫五正调）', 'union': [{'tags': ['轻音乐', '浪漫', '经典']}]}


In [8]:
similar_item = sudf.makeSimilarityBetweenItem()
similar_user = sudf.makeSimilarityBetweenUser()

/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [9]:
similar_item

,35504129,253956,1330364421,552869896,29097995,188429,26107920,26107922,1351802899,1458184212,...,1377501157,1321377770,1540081,410181618,27598835,401396,1294322,28221434,27598844,27598847
35504129,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
253956,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1330364421,0.0,0.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,...,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
552869896,0.0,0.0,0.0,1.0,0.5,0.0,0.0,1.0,0.0,0.0,...,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
29097995,0.0,0.0,0.0,0.5,1.0,0.0,0.0,0.5,0.0,0.0,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401396,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1294322,0.0,0.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,...,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
28221434,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
27598844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
similar_user

,RuRuUIH,m3NOW7Pp,UnfVc1Tg,HY0HwB,fhHz,9E,zVZzhaw,FnJi
RuRuUIH,1.000000,0.020774,0.017824,0.021934,0.016848,0.024931,0.023783,0.016895
m3NOW7Pp,0.020774,1.000000,0.023368,0.015101,0.010851,0.017282,0.014911,0.013239
UnfVc1Tg,0.017824,0.023368,1.000000,0.021886,0.020151,0.018025,0.013915,0.015873
HY0HwB,0.021934,0.015101,0.021886,1.000000,0.015831,0.015574,0.012745,0.012987
fhHz,0.016848,0.010851,0.020151,0.015831,1.000000,0.022259,0.015717,0.016744
9E,0.024931,0.017282,0.018025,0.015574,0.022259,1.000000,0.017288,0.014655
zVZzhaw,0.023783,0.014911,0.013915,0.012745,0.015717,0.017288,1.000000,0.019958
FnJi,0.016895,0.013239,0.015873,0.012987,0.016744,0.014655,0.019958,1.000000


In [11]:
# 生成用于存储的结果
# mongomus.makeRecomSongDictionForUser(
#     df_object=sudf, user_songs=user_songs)

diction = sudf.makeTopNSongs(
    # 对用户收藏的歌曲，每首获取最相似的前5首
    #  将这个数据进行几个合并，
    # 然后将数据进行随机提取，
    # 提取其中的 前5个
    similar=similar_item,
    song_cnt=5)
# 此处显示获取到每首歌的相似度前5首歌曲
print(diction)

{35504129: [591071, 28748595, 355973, 34723362, 28528279], 253956: [430297530, 32695695, 503186781, 32785761, 2346454], 1330364421: [1458021678, 1390075051, 22690147, 405167303, 1149864], 552869896: [18725436, 992891, 450041138, 26201959, 4875911], 29097995: [27598847, 501514341, 28921687, 22211516, 3954782], 188429: [1338846284, 29751585, 1471231, 26220166, 443876026], 26107920: [4348421, 18212755, 864147985, 1498556291, 28953086], 26107922: [18725436, 992891, 450041138, 26201959, 4875911], 1351802899: [1658974, 683883, 344804, 27733951, 1835745], 1458184212: [430297530, 775323, 503186781, 32785761, 440114124], 516358164: [35504129, 454698092, 553302999, 505330656, 26123237], 28377108: [1658974, 683883, 344804, 27733951, 1835745], 865460243: [1658974, 683883, 344804, 27733951, 1835745], 26484762: [27598148, 17345236, 2029777, 1818093701, 460043326], 464035867: [1338846284, 29751585, 1471231, 26220166, 443876026], 25657383: [1458021678, 1390075051, 22690147, 405167303, 1149864], 502218

In [12]:
# 这个方法获取的是用户自自身收藏的歌曲的歌曲
list_songs = mongomus.makeSongIdList(user_songs)
print(len(list_songs))
print(list_songs)

448
[1370027704, 331160, 1324244054, 19153845, 510515495, 17358564, 183679, 29119025, 24615861, 507585627, 31192913, 5027852, 554024777, 525278875, 472716484, 19078789, 36226193, 488026985, 454698089, 553901186, 407677975, 28748595, 744932, 173466, 175246, 28977397, 29133639, 1306273863, 458356832, 28959185, 447578836, 1308782735, 499165825, 1356668748, 464797841, 404166555, 5057816, 571585964, 481014963, 22774088, 1461521, 528864917, 1311535641, 22498860, 5058719, 463031, 410181618, 26123237, 679645, 27074449, 22064605, 29418350, 20781240, 591071, 35504129, 27591589, 28785381, 2003591, 1325896815, 29100034, 1801563, 31370984, 16437022, 1476449694, 636692, 27506128, 4425889, 29418323, 495857412, 1059071, 21697173, 476515542, 204217, 442898049, 479813913, 256468, 18663058, 4007275, 28889785, 1693715, 2531854, 26584453, 420513233, 28838470, 26123464, 34045202, 5168414, 17594193, 19277298, 468998937, 29712208, 1883224, 27443402, 553715430, 413834666, 29284485, 529500547, 28240590, 1887366

In [13]:
# 此处运算通过删除用户已经收藏的歌曲，然后再随机获取得到结果
list_songs_result = mongomus.makeSongsFromSimilarDict(
    diction=diction, songs=list_songs)
print(len(list_songs_result))
print(list_songs_result)

ValueError: Sample larger than population or is negative

In [ ]:
# 假设我们 LIMIT == "ALL"
#　一下是程序中启动语句
mongomus_all = mus.UserSongRecomAfterSongSimilarity()

# mongomus_all.saveUserSongRecomAnswer(
#                 mongomus_all.makeRecomAnswerSong(
#                     sign=0), update_name="SimilaritySongs", collection="song")


In [ ]:
# 获取recom中之前设置的数据
projections = {"_id": 0, "id": 1, "name": 1, "recom_users": 1}
doc = mongomus_all.rdb.findDocument(
    collection_name="recom", query=mongomus_all.query, projection=projections)
# 储存请求用户的名字
mongomus_all.usr.user_name = doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id = []
list_users_id.append(mongomus_all.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
list_users_id.extend(doc[0]['RecomUsers'])
# 构成xy轴的内容源头
docs = mongomus_all.usr.getSongsWithUserIdFromMongo(array=list_users_id)
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongomus_all.usr.makeXYMatric(docs=docs)
sudf = supd.SUPandas(datas=list(
    map(list, data.toarray())), items=mongomus_all.usr.y, users=mongomus_all.usr.x)

# 以上生成pandas 部分的代码基本一致
# 后面处理数据的方式不同，这里需要的是每首歌的相似歌曲，所以需要用户的信息


In [ ]:
# 寻找相似歌曲
dict_song = mongomus_all.makeSimilaritySongDictionForSong(
    df_object=sudf)
# 该方法中内容
# diction = df_object.makeTopNSongs(
#             # 对用户收藏的歌曲，每首获取最相似的前5首
#             # 将这个数据进行几个合并，
#             # 然后将数据进行随机提取，
#             # 提取其中的 前5个
#             similar=df_object.makeSimilarityBetweenItem(),
#             song_cnt=5)

In [ ]:
print(dict_song)